In [ ]:
# -*- coding: utf-8 -*-
"""
Section 3.4.2 - Site-level error evaluation
- Loads all saved AutoML models: automl_{5|10|20|50|170}sites_{GPP|NEE}.pkl
- Rebuilds per-site datasets with the SAME preprocessing as training
- Uses the last 10% per-site as test split (aligning with train_size=0.90)
- Computes per-site metrics: R2, RMSE, MAE, Pearson r (optional MAPE/RMSLE)
- Saves: per-model per-site CSV, overall summary CSV, RMSE boxplots for GPP/NEE
"""

import os, re, glob, sys, gc, time, shutil
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Prefer PyCaret's load_model (more stable), otherwise fall back to pickle
try:
    from pycaret.regression import load_model as pyc_load_model
except Exception:
    pyc_load_model = None
import pickle

# ========================== 0) Config (modify as needed) ==========================
MODELS_DIR = "./"  # directory containing .pkl models
MET_FOLDER = "plumber2_met_nc_files"
FLUX_FOLDER = "plumber2_nc_files"

# Features consistent with training
FEATURES_RAW = ['SWdown', 'LWdown', 'Tair', 'Qair', 'RH', 'Psurf', 'Wind',
                'CO2air', 'VPD', 'LAI', 'Ustar']
DERIVED_FEATURES = ['SW_LAI', 'RH_Tair', 'SWdown_lag1', 'Tair_lag1']
GEO_FEATURES = ['latitude', 'longitude']
FEATURES_ALL = FEATURES_RAW + DERIVED_FEATURES + GEO_FEATURES

TARGET_COLS = {"GPP": "GPP", "NEE": "NEE"}  # modify if column names differ
SITE_COL_NAME = "site"  # site name column used in exported metrics

# Training scales to be evaluated (detected from filenames, or restricted here)
SCALES_ORDER = ["5sites", "10sites", "20sites", "50sites", "170sites"]

# Output directory
OUT_DIR = Path("./eval_3_4_2_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ========================== 1) xarray open & preprocessing (same as training) ==========================
_DEF_OPEN_BACKENDS = ("netcdf4", "h5netcdf")

def _has_non_ascii(s: str) -> bool:
    try:
        s.encode('ascii')
        return False
    except UnicodeEncodeError:
        return True

def _ascii_cache_path(src_path: str) -> str:
    cache_root = Path(os.getenv('TEMP', 'C:\\temp')) / 'nc_cache'
    cache_root.mkdir(parents=True, exist_ok=True)
    return str(cache_root / Path(src_path).name)

def xr_open(path):
    abs_path = str(Path(path).resolve())
    if sys.platform.startswith('win') and _has_non_ascii(abs_path):
        cached = _ascii_cache_path(abs_path)
        try:
            if (not os.path.exists(cached)) or (os.path.getmtime(cached) < os.path.getmtime(abs_path)):
                shutil.copy2(abs_path, cached)
            read_path = cached
        except Exception:
            read_path = abs_path
    else:
        read_path = abs_path

    last_err = None
    for eng in _DEF_OPEN_BACKENDS:
        try:
            return xr.open_dataset(read_path, engine=eng)
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f"Failed to open {abs_path}, please install netCDF4/h5netcdf. Original error: {last_err}")

def _extract_scalar_value(arr) -> float:
    try:
        val = arr.values
        if np.ndim(val) == 0:
            return float(val)
        else:
            return float(np.nanmean(val))
    except Exception:
        return np.nan

def get_lat_lon(ds: xr.Dataset):
    lat_candidates = ['lat', 'latitude', 'LAT', 'Latitude']
    lon_candidates = ['lon', 'longitude', 'LON', 'Longitude']

    lat, lon = np.nan, np.nan
    for name in lat_candidates:
        if name in ds.coords:  lat = _extract_scalar_value(ds.coords[name]); break
        if name in ds.variables: lat = _extract_scalar_value(ds.variables[name]); break
    for name in lon_candidates:
        if name in ds.coords:  lon = _extract_scalar_value(ds.coords[name]); break
        if name in ds.variables: lon = _extract_scalar_value(ds.variables[name]); break

    if np.isnan(lat):
        for k in ['site_latitude', 'Latitude', 'LAT', 'lat']:
            if k in ds.attrs:
                try: lat = float(ds.attrs[k]); break
                except: pass
    if np.isnan(lon):
        for k in ['site_longitude', 'Longitude', 'LON', 'lon']:
            if k in ds.attrs:
                try: lon = float(ds.attrs[k]); break
                except: pass
    if np.isnan(lat): lat = 0.0
    if np.isnan(lon): lon = 0.0
    if lon > 180: lon -= 360.0
    return float(lat), float(lon)

def preprocess_site(met_path, flux_path, lat, lon):
    """Consistent with training, but keep time for 90/10 temporal split."""
    try:
        met_ds = xr_open(met_path)
        flux_ds = xr_open(flux_path)
        met_df = met_ds.to_dataframe().reset_index()
        flux_df = flux_ds.to_dataframe().reset_index()

        df = pd.merge_asof(
            met_df.sort_values('time'),
            flux_df.sort_values('time'),
            on='time'
        )

        keep_cols = ['time'] + FEATURES_RAW + ['GPP', 'NEE']
        df = df[keep_cols].dropna()

        df['SW_LAI'] = df['SWdown'] * df['LAI']
        df['RH_Tair'] = df['RH'] * df['Tair']
        df['SWdown_lag1'] = df['SWdown'].shift(1)
        df['Tair_lag1'] = df['Tair'].shift(1)

        df['latitude']  = np.float32(lat)
        df['longitude'] = np.float32(lon)

        df = df.dropna().sort_values('time').reset_index(drop=True)
        # Not converting to float32 here to avoid dtype sensitivity in some pipelines;
        # uncomment below if consistency is required:
        # df[df.columns] = df[df.columns].astype('float32')
        met_ds.close(); flux_ds.close()
        return df
    except Exception as e:
        print(f"❌ Preprocessing failed: {met_path} -> {e}")
        return None

# ========================== 2) Model & metrics utilities ==========================
def parse_model_filename(fname: str) -> Tuple[str, str]:
    """
    Parse (scale, target) from filename, e.g.:
      automl_20sites_GPP.pkl -> ("20sites", "GPP")
      automl_model_5sites_NEE.pkl -> ("5sites", "NEE")
      automl_gpu_20sites_GPP.pkl -> ("20sites", "GPP")
    """
    base = os.path.basename(fname).replace(".pkl", "")
    m = re.search(r"(?:automl(?:_model)?(?:_gpu)?)_(\d+sites)_(GPP|NEE)$", base, re.IGNORECASE)
    if not m:
        # fallback
        m = re.search(r"(GPP|NEE).*?(\d+sites)", base, re.IGNORECASE)
        if not m:
            raise ValueError(f"Cannot parse scale/target: {base}")
        g1, g2 = m.group(1).upper(), m.group(2).lower()
        return (g2, g1)
    return (m.group(1).lower(), m.group(2).upper())

def load_any_model(path: str):
    # Prefer PyCaret's load_model (accepts with or without .pkl)
    if pyc_load_model is not None:
        try:
            stem = path[:-4] if path.endswith(".pkl") else path
            return pyc_load_model(stem)
        except Exception:
            pass
    # Fallback to native pickle
    with open(path, "rb") as f:
        return pickle.load(f)

def safe_mape(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-6) -> float:
    denom = np.clip(np.abs(y_true), eps, None)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

def safe_rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    mask = (y_true > -1.0) & (y_pred > -1.0)
    if mask.sum() == 0:
        return float("nan")
    return float(np.sqrt(np.mean((np.log1p(y_pred[mask]) - np.log1p(y_true[mask])) ** 2)))

def compute_metrics(y_true, y_pred, target: str) -> Dict[str, float]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    rho = pearsonr(y_true, y_pred)[0] if (np.std(y_true)>1e-12 and np.std(y_pred)>1e-12) else np.nan
    out = {"MAE": mae, "RMSE": rmse, "R2": r2, "PearsonR": rho}
    # RMSLE only for GPP (since NEE may contain negative values)
    out["RMSLE"] = safe_rmsle(y_true, y_pred) if target == "GPP" else np.nan
    # MAPE can explode near zero, included as reference
    out["MAPE"] = safe_mape(y_true, y_pred)
    return out

# ========================== 3) Build evaluation dataset (per site) ==========================
def list_sites(met_folder: str) -> List[str]:
    mets = sorted(glob.glob(os.path.join(met_folder, "*_Met.nc")))
    return [os.path.basename(p).replace("_Met.nc", "") for p in mets]

def build_site_df(site_prefix: str) -> pd.DataFrame:
    met_path  = os.path.join(MET_FOLDER,  f"{site_prefix}_Met.nc")
    flux_path = os.path.join(FLUX_FOLDER, f"{site_prefix}_Flux.nc")
    if not os.path.exists(met_path) or not os.path.exists(flux_path):
        return None
    # Extract latitude/longitude
    try:
        ds = xr_open(met_path)
        lat, lon = get_lat_lon(ds)
        ds.close()
    except Exception:
        lat, lon = 0.0, 0.0
    df = preprocess_site(met_path, flux_path, lat, lon)
    if df is None or df.empty:
        return None
    # Add site name
    df[SITE_COL_NAME] = site_prefix
    return df

def per_site_time_split(df_site: pd.DataFrame, test_ratio=0.10) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Split chronologically into 90/10, returning (train, test). Only test is used for evaluation here."""
    df_site = df_site.sort_values("time").reset_index(drop=True)
    n = len(df_site)
    k = max(1, int((1.0 - test_ratio) * n))
    return df_site.iloc[:k, :], df_site.iloc[k:, :]

# ========================== 4) Main evaluation workflow ==========================
def evaluate_model_on_all_sites(model_path: str, sites: List[str]) -> pd.DataFrame:
    scale, target = parse_model_filename(model_path)
    target_col = TARGET_COLS[target]
    model = load_any_model(model_path)

    records = []
    for site in sites:
        df_site = build_site_df(site)
        if df_site is None or df_site.empty:
            print(f"[Skip] {site}: failed to build data")
            continue

        # Temporal split: last 10% as test
        _, test_df = per_site_time_split(df_site, test_ratio=0.10)
        if test_df.empty:
            continue

        # Features & target
        X_test = test_df[FEATURES_ALL].copy()
        y_test = test_df[target_col].values

        # Prediction
        try:
            y_pred = model.predict(X_test)
        except Exception as e:
            print(f"[WARN] Prediction failed {os.path.basename(model_path)} @ {site}: {e}")
            continue

        metrics = compute_metrics(y_test, y_pred, target)
        rec = {
            "model_file": os.path.basename(model_path),
            "scale": scale, "target": target, "site": site,
            "lat": float(test_df["latitude"].iloc[0]),
            "lon": float(test_df["longitude"].iloc[0])
        }
        rec.update(metrics)
        records.append(rec)

    df_res = pd.DataFrame(records)
    if not df_res.empty:
        out_csv = OUT_DIR / f"site_metrics_{scale}_{target}.csv"
        df_res.to_csv(out_csv, index=False)
        print(f"Saved: {out_csv}")
    return df_res

def plot_rmse_boxplots(all_df: pd.DataFrame, target: str):
    df_t = all_df[all_df["target"] == target].copy()
    if df_t.empty:
        return
    order = [s for s in SCALES_ORDER if s in df_t["scale"].unique()]
    df_t["scale"] = pd.Categorical(df_t["scale"], categories=order, ordered=True)
    plt.figure(figsize=(7, 4.5))
    df_t.boxplot(column="RMSE", by="scale", grid=False)
    plt.title(f"Per-site RMSE across training scales ({target})")
    plt.suptitle("")
    plt.xlabel("Training scale")
    plt.ylabel("RMSE")
    out_png = OUT_DIR / f"boxplot_rmse_{target}.png"
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()
    print(f"Saved: {out_png}")

def main():
    print(f"Working dir: {os.getcwd()}")
    model_paths = sorted(glob.glob(os.path.join(MODELS_DIR, "*.pkl")))
    keep = []
    for p in model_paths:
        try:
            parse_model_filename(p)
            keep.append(p)
        except Exception:
            pass
    if not keep:
        raise RuntimeError("No parsable .pkl models found. Please ensure filenames like automl_20sites_GPP.pkl")

    # List available sites
    sites = list_sites(MET_FOLDER)
    if not sites:
        raise RuntimeError("No *_Met.nc found, please check MET_FOLDER path.")

    all_list = []
    for mp in keep:
        print(f"\n=== Evaluating {os.path.basename(mp)} ===")
        t0 = time.time()
        df_res = evaluate_model_on_all_sites(mp, sites)
        if df_res is not None and not df_res.empty:
            all_list.append(df_res)
        print(f"Elapsed: {time.time()-t0:.1f}s")

    if not all_list:
        print("No results produced.")
        return

    all_df = pd.concat(all_list, ignore_index=True)
    all_df.to_csv(OUT_DIR / "all_site_metrics_all_models.csv", index=False)
    print(f"Saved: {OUT_DIR/'all_site_metrics_all_models.csv'}")

    # Summary: by (target, scale)
    summary = (all_df.groupby(["target", "scale"])
                      .agg(n_sites=("site","nunique"),
                           MAE_mean=("MAE","mean"),
                           RMSE_mean=("RMSE","mean"),
                           R2_mean=("R2","mean"),
                           PearsonR_mean=("PearsonR","mean"))
                      .reset_index())
    summary.to_csv(OUT_DIR / "summary_by_scale_target.csv", index=False)
    print(f"Saved: {OUT_DIR/'summary_by_scale_target.csv'}")

    # Boxplots (for Section 3.4.2)
    plot_rmse_boxplots(all_df, "GPP")
    plot_rmse_boxplots(all_df, "NEE")

if __name__ == "__main__":
    main()


Working dir: /root/autodl-tmp/dataset

=== Evaluating automl_10sites_GPP.pkl ===
Transformation Pipeline and Model Successfully Loaded
Saved: eval_3_4_2_outputs/site_metrics_10sites_GPP.csv
Elapsed: 69.7s

=== Evaluating automl_10sites_NEE.pkl ===
Transformation Pipeline and Model Successfully Loaded
